In [75]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

# Custom functions to create visual charts
from chart_utils import TimeSeriesImageDataset, create_area_chart, create_bar_chart, accuracy_fn

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Read UCR Dataset
def read_ucr(filename):
    data = []
    labels = []
    
    with open(filename, 'r') as file:
        for line in file:
            parts = line.strip().split(',')
            if len(parts) < 2:  # Ensure there's at least one feature and one label
                continue
            features = [float(f) for f in parts[:-1]]
            label = int(parts[-1].split(':')[-1])  # Handle label after the colon
            data.append(features)
            labels.append(label)
    
    print(f"Loaded {len(data)} samples from {filename}")
    return np.array(data), np.array(labels)

# File paths
train_file = 'ECG/ECG_TRAIN.ts'
test_file = 'ECG/ECG_TEST.ts'

# Load dataset
x_train, y_train = read_ucr(train_file)
x_test, y_test = read_ucr(test_file)

# Normalize labels to be within range [0, num_classes-1]
unique_labels = np.unique(y_train)
label_map = {label: idx for idx, label in enumerate(unique_labels)}
y_train = np.array([label_map[label] for label in y_train])
y_test = np.array([label_map[label] for label in y_test])

nb_classes = len(unique_labels)

# Verify labels are within range
print(f"Number of classes: {nb_classes}")
print(f"y_train unique labels: {np.unique(y_train)}")
print(f"y_test unique labels: {np.unique(y_test)}")

# Ensure labels are within the expected range [0, num_classes-1]
assert y_train.min() >= 0 and y_train.max() < nb_classes, "Train labels are out of range"
assert y_test.min() >= 0 and y_test.max() < nb_classes, "Test labels are out of range"

# Normalize features
x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_train = (x_train - x_train_mean) / x_train_std
x_test = (x_test - x_train_mean) / x_train_std

# Create dataset and dataloader
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)
])

train_dataset = TimeSeriesImageDataset(x_train, y_train, transform)
test_dataset = TimeSeriesImageDataset(x_test, y_test, transform)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define CNN
class CNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNN, self).__init__()
        self.layer_stack = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.layer_stack(x)

# Example usage of the model
input_shape = (6, 128, 128)  # Adjust input shape to reflect the combined channel count
num_classes = nb_classes  # Number of output classes

model = CNN(input_shape[0], num_classes)
print(model)

def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

def train_model(model, train_dataloader, test_dataloader, criterion, optimizer, scheduler, num_epochs, device):
    model.to(device)

    for epoch in range(num_epochs):
        ### Training
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images_area, images_bar, labels in train_dataloader:
            images_area, images_bar, labels = images_area.to(device), images_bar.to(device), labels.to(device).long()

            # Combine area and bar charts along the channel dimension
            combined_images = torch.cat((images_area, images_bar), dim=1)

            optimizer.zero_grad()
            outputs = model(combined_images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_dataloader)
        epoch_accuracy = 100 * correct / total

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

        ### Testing
        model.eval()
        test_loss = 0.0
        correct = 0
        total = 0

        with torch.inference_mode():
            for images_area, images_bar, labels in test_dataloader:
                images_area, images_bar, labels = images_area.to(device), images_bar.to(device), labels.to(device).long()

                # Combine area and bar charts along the channel dimension
                combined_images = torch.cat((images_area, images_bar), dim=1)

                outputs = model(combined_images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        test_loss /= len(test_dataloader)
        test_accuracy = 100 * correct / total

        print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

        # Step the scheduler
        scheduler.step(test_loss)

# Define the model, criterion, optimizer, scheduler, and other parameters
input_shape = (6, 128, 128)  # Adjust input shape to reflect the combined channel count
num_classes = nb_classes  # Number of output classes

model = CNN(input_shape[0], num_classes)  # Adjust this line according to your model's class name

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

# Number of epochs
num_epochs = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train and evaluate the model
train_model(model, train_dataloader, test_dataloader, criterion, optimizer, scheduler, num_epochs, device)


Loaded 100 samples from ECG/ECG_TRAIN.ts
Loaded 100 samples from ECG/ECG_TEST.ts
Number of classes: 2
y_train unique labels: [0 1]
y_test unique labels: [0 1]
CNN(
  (layer_stack): Sequential(
    (0): Conv2d(6, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Fl

KeyboardInterrupt: 